## 텍스트 전처리

판결 요지 부분에서 불용어 및 특수기호를 제거한 뒤 정규화 된 띄어쓰기를 사용하기 위한 교정을 진행합니다.

In [19]:
import pandas as pd
import re
from konlpy.tag import Okt

In [20]:
# 기존 kobert 모델링에서 사용한 데이터 일부 사용
dataframe = pd.read_excel("../data-preprocessing-kobert/output/03_data_output.xlsx", engine='openpyxl')

In [21]:
okt = Okt()
filtered_judgement_reason = []

# 정확한 뛰어쓰기를 위해 추가
def spacing_okt(wrongSentence):
    tagged = okt.pos(wrongSentence)
    corrected = ""
    for i in tagged:
        if i[1] in ('Josa', 'PreEomi', 'Eomi', 'Suffix', 'Punctuation'):
            corrected += i[0]
        else:
            corrected += " "+i[0]
    if corrected[0] == " ":
        corrected = corrected[1:]
    return corrected

# 텍스트 전처리
for i, row in dataframe.iterrows():
    if i % 1000 == 0:
        print("Step ", i)

    reason = row['judgement_reason']
    reason = reason.replace(',','').replace('"','').replace('\'','').replace('.','').replace('(',' ').\
            replace(')','').replace('!','').replace('?','').replace(':','').replace(';','').lower()
    reason = reason.replace("\n"," ")
    reason = re.sub(r'\d+?\.\s\d+\.\s\d+\.', '', reason) # 날짜 제거
    reason = re.sub(r'\b\w+법원|지원', '', reason) # 법원명 제거
    reason = re.sub('수사보고|범 죄 사 실|범죄사실', '', reason) # 판례의 기본적인 문구 제거
    reason = re.sub(r'[「」『』\[\],.:%○]', '', reason) # 특수기호 제거
    
    filtered_judgement_reason.append(spacing_okt(reason))

dataframe['filtered_judgement_reason'] = filtered_judgement_reason


Step  0
Step  1000
Step  2000
Step  3000
Step  4000
Step  5000
Step  6000
Step  7000
Step  8000
Step  9000
Step  10000
Step  11000
Step  12000
Step  13000
Step  14000
Step  15000
Step  16000
Step  17000
Step  18000
Step  19000
Step  20000


In [24]:
dataframe.to_excel("./output/01-text-preprocessing.xlsx", engine="openpyxl")